In [1]:
!pip install gudhi
!pip install --upgrade category_encoders
!pip install -U giotto-tda
!pip install PyPDF2
!pip install fpdf



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 29.3 MB 81.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 4.5 MB/s 
     |████████████████████████████████| 404 kB 16.9 MB/s 
     |████████████████████████████████| 3.1 MB 38.9 MB/s 
  Created wheel for python-igraph: filename=python_igraph-0.9.11-py3-none-any.whl size=9074 sha256=da07d4a65ff63d3aea21e7a67869103f060bde86b7f7ac06b4472a08fe751c2f
  Stored in directory: /root/.cache/pip/wheels/7d/e4/0c/0c83f70bd0b99ce8aea47a21f8e52502169e7dd17808d12f30
Successfully built python-igraph
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████

# 1. Load Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1.1 Load Target values

In the target-dataset, we have 1206 patients(1206 rows in the dataset) and 583 different targets(583 different target values). If we build a machine learning model, we can predict these 583 different targets. The target selection is based on us.

In [3]:
#Load the target file to Jupyter Notebook

import pandas as pd
import numpy as np

df_struct = pd.read_csv('/content/drive/MyDrive/GUDHI-tda-tutorials-mapper/df_struct.csv')
print(df_struct.shape)

(1206, 583)


In [4]:
#Let's see what kind of targets we have:
df_struct.columns

Index(['Unnamed: 0', 'Subject', 'Release', 'Acquisition', 'Gender', 'Age',
       '3T_Full_MR_Compl', 'T1_Count', 'T2_Count', '3T_RS-fMRI_Count',
       ...
       'Noise_Comp', 'Odor_Unadj', 'Odor_AgeAdj', 'PainIntens_RawScore',
       'PainInterf_Tscore', 'Taste_Unadj', 'Taste_AgeAdj', 'Mars_Log_Score',
       'Mars_Errs', 'Mars_Final'],
      dtype='object', length=583)

## 1.2 Load DTI connectivity matrices of 998 patients:

Our goal is to predict targets given by connectivity matrices. These connectivity matrices represent neural relationships of different brain regions. We have 998 connectivity matrices so we have 998 patients.

In [5]:
# Let's now load it into a list of 998 pandas DataFrames
import pickle
open_file = open('/content/drive/MyDrive/GUDHI-tda-tutorials-mapper/DTI.txt', "rb")
loaded_DTI = pickle.load(open_file)
open_file.close()

In [6]:
#Just to check that we 998 connectivity matrixes
len(loaded_DTI), type(loaded_DTI)

(998, list)

The connectivity matrices have shape (116,116). That means we have 116 brain regions.

In [7]:
loaded_DTI[0].to_numpy().shape

(116, 116)

## 1.3 Load IDs of 998 patient:

In the target-dataset we had 1206 patients, however we have 998 connectivity matrices. So we need to select 998 patients' target values. How to do it? By matching the IDs of patients. First we load the patient IDs:

In [8]:
# You may need the IDs of all DTI networks in your project. We give this for you here:
#!wget -O 'IDsDTI.txt' https://www.dropbox.com/s/k7wuffrr0e26a7m/IDsDTI.txt?dl=0
open_file = open('/content/drive/MyDrive/GUDHI-tda-tutorials-mapper/IDsDTI.txt', "rb")
DTI_IDs = pickle.load(open_file)
open_file.close()

In [9]:
len(DTI_IDs)

998

In [10]:
DTI_IDs = pd.DataFrame(DTI_IDs,columns =['Subject'])
DTI_IDs.head()

,Subject
0,120212
1,108222
2,111009
3,393247
4,211821


## 1.3 Load Brain region names:

We may also need the 116 brain region names in future so we load it too:


In [11]:
# You may need the list of names
#!wget -O 'AAL_names.txt' https://www.dropbox.com/s/5fkp3s5suyibe57/AAL_names.txt?dl=0

In [12]:
open_file = open('/content/drive/MyDrive/GUDHI-tda-tutorials-mapper/AAL_names.txt', "rb")
loaded_AAL_names = pickle.load(open_file)
open_file.close()
len(loaded_AAL_names)

116

In [13]:
#loaded_AAL_names

# 2. Preprocessing Data

### 2.1 Match ID's with target values:

We do mathing the patient IDs with the target values here. The 'Subject' column in both datasets represent the IDs.

In [14]:
df_targets = df_struct.copy()

In [15]:
df_targets = df_struct.copy()

for i in range(len(df_targets)):
    
    flag=0
    
    for j in range(len(DTI_IDs)):
        if int(DTI_IDs['Subject'][j])== df_targets['Subject'][i]:
            flag = 1
        
    if flag==0:
        df_targets = df_targets.drop([i])
        

In [16]:
df_targets.shape

(998, 583)

### 2.2 Take tranpose to make matrices symmetric

Our connectivity matrices are not symmetric. The DTI_ab matrix measures the amount of fibers from a to b and DTI_ba from b to a, which may differ. Simply, we have a directed graph, and weights are in two opposite dimension. We need to make the matrice symmetric:

In [17]:
DTI_sym = loaded_DTI.copy()

In [18]:
loaded_DTI[0]

,Precentral_L,Precentral_R,Frontal_Sup_L,Frontal_Sup_R,Frontal_Sup_Orb_L,Frontal_Sup_Orb_R,Frontal_Mid_L,Frontal_Mid_R,Frontal_Mid_Orb_L,Frontal_Mid_Orb_R,...,Cerebelum_10_L,Cerebelum_10_R,Vermis_1_2,Vermis_3,Vermis_4_5,Vermis_6,Vermis_7,Vermis_8,Vermis_9,Vermis_10
data.1,,,,,,,,,,,,,,,,,,,,,
Precentral_L,0.000000,0.111284,0.244940,0.036833,0.000000,0.000000,0.316256,0.031119,0.000000,0.037296,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Precentral_R,0.058161,0.000000,0.024979,0.077539,0.000000,0.000000,0.003282,0.193988,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Frontal_Sup_L,0.638208,0.124529,0.000000,0.278553,0.276687,0.012743,1.000000,0.275736,0.078628,0.029371,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Frontal_Sup_R,0.077435,0.311908,0.224755,0.000000,0.014218,1.000000,0.040737,1.000000,0.024073,0.350583,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Frontal_Sup_Orb_L,0.000000,0.000000,0.031933,0.002034,0.000000,0.000000,0.002370,0.000000,0.542085,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vermis_6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.031301,0.237843,0.366248,0.000000,0.432853,0.124460,0.226047,1.000000
Vermis_7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.043103,0.232500,0.000000,0.446245,0.076638,0.506090,0.000000,0.855930,0.774257,0.206943
Vermis_8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.040640,0.415833,0.000000,0.473040,0.080923,0.134490,0.791066,0.000000,0.986443,0.269161


In [19]:
#Make the matrices symmetric

for i in range(len(loaded_DTI)):
    DTI_sym[i] = (DTI_sym[i] + DTI_sym[i].T)/2 
DTI_sym[0]

,Precentral_L,Precentral_R,Frontal_Sup_L,Frontal_Sup_R,Frontal_Sup_Orb_L,Frontal_Sup_Orb_R,Frontal_Mid_L,Frontal_Mid_R,Frontal_Mid_Orb_L,Frontal_Mid_Orb_R,...,Cerebelum_10_L,Cerebelum_10_R,Vermis_1_2,Vermis_3,Vermis_4_5,Vermis_6,Vermis_7,Vermis_8,Vermis_9,Vermis_10
data.1,,,,,,,,,,,,,,,,,,,,,
Precentral_L,0.000000,0.084723,0.441574,0.057134,0.000000,0.000000,0.570140,0.037480,0.000000,0.021363,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Precentral_R,0.084723,0.000000,0.074754,0.194723,0.000000,0.000000,0.009822,0.358454,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Frontal_Sup_L,0.441574,0.074754,0.000000,0.251654,0.154310,0.007062,1.000000,0.212413,0.045227,0.015506,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Frontal_Sup_R,0.057134,0.194723,0.251654,0.000000,0.008126,0.567128,0.045612,0.835060,0.014280,0.187429,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Frontal_Sup_Orb_L,0.000000,0.000000,0.154310,0.008126,0.000000,0.000000,0.011454,0.000000,0.624236,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vermis_6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.020406,0.178908,0.496907,0.000000,0.469472,0.129475,0.236835,0.685049
Vermis_7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.029044,0.136159,0.000000,0.319384,0.094477,0.469472,0.000000,0.823498,0.749837,0.136225
Vermis_8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.027964,0.246444,0.000000,0.346928,0.104622,0.129475,0.823498,0.000000,0.993222,0.180674


In [20]:
len(DTI_sym), type(DTI_sym), type(DTI_sym[0])

(998, list, pandas.core.frame.DataFrame)

### 2.4 Feature Extraction w/ SparseRipsPersistence

In [21]:
#!python -m pip install -U giotto-tda

In [22]:
"""
import numpy as np
from numpy.random import default_rng
rng = default_rng(42)  # Create a random number generator

from scipy.spatial.distance import pdist, squareform
from scipy.sparse import coo_matrix

from gtda.graphs import GraphGeodesicDistance
from gtda.homology import VietorisRipsPersistence, SparseRipsPersistence, FlagserPersistence

from igraph import Graph

from IPython.display import SVG, display
"""

'\nimport numpy as np\nfrom numpy.random import default_rng\nrng = default_rng(42)  # Create a random number generator\n\nfrom scipy.spatial.distance import pdist, squareform\nfrom scipy.sparse import coo_matrix\n\nfrom gtda.graphs import GraphGeodesicDistance\nfrom gtda.homology import VietorisRipsPersistence, SparseRipsPersistence, FlagserPersistence\n\nfrom igraph import Graph\n\nfrom IPython.display import SVG, display\n'

In [23]:
"""
symmetric_DTI_np = []
nonsymmetric_DTI_np = []
for i in range(len(DTI_sym)):
    symmetric_DTI_np.append(DTI_sym[i].to_numpy()) 
    nonsymmetric_DTI_np.append(loaded_DTI[i].to_numpy())
    
len(symmetric_DTI_np), len(nonsymmetric_DTI_np)
"""

'\nsymmetric_DTI_np = []\nnonsymmetric_DTI_np = []\nfor i in range(len(DTI_sym)):\n    symmetric_DTI_np.append(DTI_sym[i].to_numpy()) \n    nonsymmetric_DTI_np.append(loaded_DTI[i].to_numpy())\n    \nlen(symmetric_DTI_np), len(nonsymmetric_DTI_np)\n'

In [24]:
#!pip install -U giotto-tda

In [25]:
#Plotting:
#from gtda.plotting import plot_diagram
#i = 0
#plot_diagram(diagrams[i])

#### SparseRipsPersistence - Symmetric

In [26]:
"""
# Instantiate topological transformer
SP = SparseRipsPersistence( homology_dimensions=[0, 1, 2])

# Compute persistence diagrams corresponding to each entry
diagrams_sp_sym = SP.fit_transform(symmetric_DTI_np)

print(f"diagrams.shape: {diagrams_sp_sym.shape} ({diagrams_sp_sym.shape[1]} topological features)")
"""

'\n# Instantiate topological transformer\nSP = SparseRipsPersistence( homology_dimensions=[0, 1, 2])\n\n# Compute persistence diagrams corresponding to each entry\ndiagrams_sp_sym = SP.fit_transform(symmetric_DTI_np)\n\nprint(f"diagrams.shape: {diagrams_sp_sym.shape} ({diagrams_sp_sym.shape[1]} topological features)")\n'

In [27]:
"""
from gtda.diagrams import PersistenceEntropy

PE = PersistenceEntropy()

features_sp_sym = PE.fit_transform(diagrams_sp_sym)
features_sp_sym
"""

'\nfrom gtda.diagrams import PersistenceEntropy\n\nPE = PersistenceEntropy()\n\nfeatures_sp_sym = PE.fit_transform(diagrams_sp_sym)\nfeatures_sp_sym\n'

Save the Topological features into a .csv file

In [28]:
"""
# save numpy array as csv file
from numpy import asarray
from numpy import savetxt
# save to csv file
savetxt('SparseRipsPersistence_dim012_sym.csv', features_sp_sym, delimiter=',')

"""

"\n# save numpy array as csv file\nfrom numpy import asarray\nfrom numpy import savetxt\n# save to csv file\nsavetxt('SparseRipsPersistence_dim012_sym.csv', features_sp_sym, delimiter=',')\n\n"

Load topological features coming from SparseRips Complex:

In [29]:
# load numpy array from csv file
from numpy import loadtxt
# load array
loaded_features_sp_sym = loadtxt('/content/drive/MyDrive/GUDHI-tda-tutorials-mapper/SparseRipsPersistence_dim01_sym.csv', delimiter=',')
# print the array
print(loaded_features_sp_sym)
type(loaded_features_sp_sym)

[[6.81966661 4.52649199]
 [6.82150911 4.72372002]
 [6.81782604 4.45223201]
 ...
 [6.82301216 4.77680341]
 [6.8243857  4.41155091]
 [6.82064777 4.28775706]]


numpy.ndarray

Convert TDA features to a pandas df and add Gender column.

In [30]:
df_tda = pd.DataFrame(loaded_features_sp_sym, columns=["TDA_feature_1","TDA_feature_2"])  #,"TDA_feature_3"])
df_tda.head(5)

,TDA_feature_1,TDA_feature_2
0,6.819667,4.526492
1,6.821509,4.723720
2,6.817826,4.452232
3,6.820391,4.500798
4,6.824610,4.486151


### 3- Labels and Meanings

In [31]:
#COGNITION
Cognition_meanings2=[
                        "PicSeq_AgeAdj" , "Episodic Memory",
                         "CardSort_AgeAdj","Executive Function/ Cognitive Flexibility",
                         "Flanker_AgeAdj","Executive Function/ Inhibition",
                         
                         "PMAT24_A_CR","Fluid Intelligence",
                         "PMAT24_A_SI", "Fluid Intelligence",
                          "PMAT24_A_RTCR","Fluid Intelligence",
                         
                        "ReadEng_AgeAdj",   "Language/Reading Decoding ",
                        "PicVocab_AgeAdj",  "Language/Vocabulary Comprehension",
                        "ProcSpeed_AgeAdj", "Processing Speed" ,
                         
                         "DDisc_AUC_200","DDisc_AUC_40K", "Self-regulation/Impulsivity (Delay Discounting)",
                         "DDisc_SV_1mo_200", "Self-regulation/Impulsivity (Delay Discounting)",
                          "DDisc_SV_6mo_200","Self-regulation/Impulsivity (Delay Discounting)",
                          "DDisc_SV_1yr_200","Self-regulation/Impulsivity (Delay Discounting)",
                          "DDisc_SV_3yr_200","Self-regulation/Impulsivity (Delay Discounting)",
                          "DDisc_SV_5yr_200","Self-regulation/Impulsivity (Delay Discounting)",
                          "DDisc_SV_10yr_200", "Self-regulation/Impulsivity (Delay Discounting)",
                         "DDisc_SV_1mo_40K", "Self-regulation/Impulsivity (Delay Discounting)",
                          "DDisc_SV_6mo_40K","Self-regulation/Impulsivity (Delay Discounting)",
                        "DDisc_SV_1yr_40K","Self-regulation/Impulsivity (Delay Discounting)",
                        "DDisc_SV_3yr_40K","Self-regulation/Impulsivity (Delay Discounting)",
                        "DDisc_SV_5yr_40K","Self-regulation/Impulsivity (Delay Discounting)",
                          "DDisc_SV_10yr_40K","Self-regulation/Impulsivity (Delay Discounting)",

                        "VSPLOT_TC",    "Spatial Orientation",
                         "VSPLOT_CRTE","Spatial Orientation",
                         "VSPLOT_OFF","Spatial Orientation",

                         "SCPT_SEN","SCPT_SPEC", "Sustained Attention (Short Penn Continuous Performance Test)",
                         "SCPT_TP","Sustained Attention (Short Penn Continuous Performance Test)",
                     "SCPT_TN","Sustained Attention (Short Penn Continuous Performance Test)",
                    "SCPT_FP","Sustained Attention (Short Penn Continuous Performance Test)",
                    "SCPT_FN","Sustained Attention (Short Penn Continuous Performance Test)",
                    "SCPT_TPRT","Sustained Attention (Short Penn Continuous Performance Test)",
                    "SCPT_LRNR","Sustained Attention (Short Penn Continuous Performance Test)",
                         
                         
                         "IWRD_TOT",        "Verbal Episodic Memory",
                         "IWRD_RTC","Verbal Episodic Memory",
                    
                         "ListSort_AgeAdj", "Working Memory (List Sorting)"
                        ]  
#COGNITION
Cognition_meanings=[
                        "PicSeq_AgeAdj",   "COG_Episodic Memory-PicSeq_AgeAdj",
                         "CardSort_AgeAdj", "COG_Executive Function/ Cognitive Flexibility - CardSort_AgeAdj",
                         "Flanker_AgeAdj",  "COG_Executive Function/ Inhibition - Flanker_AgeAdj",
                         
                         "PMAT24_A_CR",     "COG_Fluid Intelligence-PMAT24_A_CR",
                         #"PMAT24_A_SI", "PMAT24_A_RTCR",
                         
                        "ReadEng_AgeAdj",   "COG_Language/Reading Decoding-ReadEng_AgeAdj" ,
                        "PicVocab_AgeAdj",  "COG_Language/Vocabulary Comprehension-PicVocab_AgeAdj",
                        "ProcSpeed_AgeAdj", "COG_Processing Speed -ProcSpeed_AgeAdj",
                         
                         "DDisc_AUC_200", "COG_Self-regulation/Impulsivity (Delay Discounting)-DDisc_AUC_200",
                    "DDisc_AUC_40K", "COG_Self-regulation/Impulsivity (Delay Discounting)-DDisc_AUC_40K",
                         #"DDisc_SV_1mo_200", "DDisc_SV_6mo_200","DDisc_SV_1yr_200","DDisc_SV_3yr_200","DDisc_SV_5yr_200","DDisc_SV_10yr_200", 
                         #"DDisc_SV_1mo_40K", "DDisc_SV_6mo_40K","DDisc_SV_1yr_40K","DDisc_SV_3yr_40K","DDisc_SV_5yr_40K","DDisc_SV_10yr_40K",

                        "VSPLOT_TC",        "COG_Spatial Orientation-VSPLOT_TC",
                         #"VSPLOT_CRTE",
                         #"VSPLOT_OFF",


                         "SCPT_SEN", "COG_Sustained Attention-SCPT_SEN",
                    "SCPT_SPEC", "COG_Sustained Attention-SCPT_SPEC" ,#"Sustained Attention" #(Short Penn Continuous Performance Test)
                         #"SCPT_TP", #"SCPT_TN","SCPT_FP","SCPT_FN","SCPT_TPRT","SCPT_LRNR",
                         
                         
                         "IWRD_TOT",        "COG_Verbal Episodic Memory-IWRD_TOT",
                         #"IWRD_RTC",
                         "ListSort_AgeAdj", "COG_Working Memory (List Sorting)-ListSort_AgeAdj",
                        ]  

In [32]:
#EMOTION
Emotion_meanings=[
                       "ER40_CR", "EMO_Emotion Recognition (Penn Emotion Recognition Test)-ER40_CR",
                       #"ER40_CRT","ER40ANG","ER40FEAR","ER40HAP","ER40NOE","ER40SAD",

                      "AngHostil_Unadj",  "EMO_Negative Affect (Sadness, Fear, Anger)-AngHostil_Unadj",
                      "AngAffect_Unadj",  "EMO_Negative Affect (Sadness, Fear, Anger)-AngAffect_Unadj",
                  "AngAggr_Unadj", "EMO_Negative Affect (Sadness, Fear, Anger)-AngAggr_Unadj",
                  "FearAffect_Unadj", "EMO_Negative Affect (Sadness, Fear, Anger)-FearAffect_Unadj",
                      "FearSomat_Unadj", "EMO_Negative Affect (Sadness, Fear, Anger)-FearSomat_Unadj",
                      "Sadness_Unadj", "EMO_Negative Affect (Sadness, Fear, Anger)-Sadness_Unadj",
                       
                       "LifeSatisf_Unadj","EMO_Psychological Well-being -LifeSatisf_Unadj",
                       "MeanPurp_Unadj","EMO_Psychological Well-being-MeanPurp_Unadj ",
                  "PosAffect_Unadj","EMO_Psychological Well-being -PosAffect_Unadj",

                        "Loneliness_Unadj","EMO_Social Relationships -Loneliness_Unadj",
                       "Friendship_Unadj","EMO_Social Relationships -Friendship_Unadj",
                  "PercHostil_Unadj","EMO_Social Relationships -PercHostil_Unadj",
                       "PercReject_Unadj","EMO_Social Relationships -PercReject_Unadj",
                  "EmotSupp_Unadj","EMO_Social Relationships -EmotSupp_Unadj",
                  "InstruSupp_Unadj","EMO_Social Relationships -InstruSupp_Unadj",

                       "PercStress_Unadj", "EMO_Stress and Self Efficacy -PercStress_Unadj",
                       "SelfEff_Unadj","EMO_Stress and Self Efficacy -SelfEff_Unadj",

                       
]


In [33]:
#MOTOR 
Motor_meanings=[
                     
                     "Endurance_AgeAdj", "MOT_Endurance (2 minute walk test)-Endurance_AgeAdj",
                     "GaitSpeed_Comp", "MOT_Locomotion (4-meter walk test)-GaitSpeed_Comp",
                     "Dexterity_AgeAdj", "MOT_Dexterity (9-hole Pegboard)-Dexterity_AgeAdj",
                     "Strength_AgeAdj", "MOT_Strength (Grip Strength Dynamometry)-Strength_AgeAdj"          
]

In [34]:
#OTHER 
Other_meanings=[
                     "Age", "OTH_Age",
                    "Gender", "OTH_Gender",
                "TDA_feature_1", "OTH_TDA_feature_1",
                "TDA_feature_2","OTH_TDA_feature_2"
                     
]

In [35]:
def make_dict(lists):
  mydic={}
  for i in range(0,len(lists),2):
    mydic[lists[i]]=lists[i+1]
  return mydic

all_meanings = Motor_meanings + Cognition_meanings + Emotion_meanings + Other_meanings
print(len(all_meanings))
meanings_dictionary = make_dict(all_meanings)
print(meanings_dictionary)


80
{'Endurance_AgeAdj': 'MOT_Endurance (2 minute walk test)-Endurance_AgeAdj', 'GaitSpeed_Comp': 'MOT_Locomotion (4-meter walk test)-GaitSpeed_Comp', 'Dexterity_AgeAdj': 'MOT_Dexterity (9-hole Pegboard)-Dexterity_AgeAdj', 'Strength_AgeAdj': 'MOT_Strength (Grip Strength Dynamometry)-Strength_AgeAdj', 'PicSeq_AgeAdj': 'COG_Episodic Memory-PicSeq_AgeAdj', 'CardSort_AgeAdj': 'COG_Executive Function/ Cognitive Flexibility - CardSort_AgeAdj', 'Flanker_AgeAdj': 'COG_Executive Function/ Inhibition - Flanker_AgeAdj', 'PMAT24_A_CR': 'COG_Fluid Intelligence-PMAT24_A_CR', 'ReadEng_AgeAdj': 'COG_Language/Reading Decoding-ReadEng_AgeAdj', 'PicVocab_AgeAdj': 'COG_Language/Vocabulary Comprehension-PicVocab_AgeAdj', 'ProcSpeed_AgeAdj': 'COG_Processing Speed -ProcSpeed_AgeAdj', 'DDisc_AUC_200': 'COG_Self-regulation/Impulsivity (Delay Discounting)-DDisc_AUC_200', 'DDisc_AUC_40K': 'COG_Self-regulation/Impulsivity (Delay Discounting)-DDisc_AUC_40K', 'VSPLOT_TC': 'COG_Spatial Orientation-VSPLOT_TC', 'SCPT_S

In [36]:
meanings_dictionary["VSPLOT_TC"]

'COG_Spatial Orientation-VSPLOT_TC'

#3. Select columns:

In [37]:
#COGNITION
Cognition_feature_names=[
                        "PicSeq_AgeAdj",   #Episodic Memory
                         "CardSort_AgeAdj", #Executive Function/ Cognitive Flexibility
                         "Flanker_AgeAdj",  #Executive Function/ Inhibition
                         
                         "PMAT24_A_CR",     #Fluid Intelligence
                         #"PMAT24_A_SI", "PMAT24_A_RTCR",
                         
                        "ReadEng_AgeAdj",   #Language/Reading Decoding 
                        "PicVocab_AgeAdj",  #Language/Vocabulary Comprehension
                        "ProcSpeed_AgeAdj", #Processing Speed 
                         
                         "DDisc_AUC_200","DDisc_AUC_40K", #Self-regulation/Impulsivity (Delay Discounting)
                         #"DDisc_SV_1mo_200", "DDisc_SV_6mo_200","DDisc_SV_1yr_200","DDisc_SV_3yr_200","DDisc_SV_5yr_200","DDisc_SV_10yr_200", 
                         #"DDisc_SV_1mo_40K", "DDisc_SV_6mo_40K","DDisc_SV_1yr_40K","DDisc_SV_3yr_40K","DDisc_SV_5yr_40K","DDisc_SV_10yr_40K",

                        "VSPLOT_TC",        #Spatial Orientation
                         #"VSPLOT_CRTE",
                         #"VSPLOT_OFF",

                         "SCPT_SEN","SCPT_SPEC", #Sustained Attention (Short Penn Continuous Performance Test)
                         #"SCPT_TP", #"SCPT_TN","SCPT_FP","SCPT_FN","SCPT_TPRT","SCPT_LRNR",
                         
                         
                         "IWRD_TOT",        #Verbal Episodic Memory
                         #"IWRD_RTC",
                         "ListSort_AgeAdj", #Working Memory (List Sorting)
                        ]  


In [38]:
#EMOTION
Emotion_feature_names=[
                       "ER40_CR", #Emotion Recognition (Penn Emotion Recognition Test)
                       #"ER40_CRT","ER40ANG","ER40FEAR","ER40HAP","ER40NOE","ER40SAD",

                      "AngHostil_Unadj",  #Negative Affect (Sadness, Fear, Anger)
                      #"AngAffect_Unadj","AngAggr_Unadj","FearAffect_Unadj",
                      "FearSomat_Unadj",
                      "Sadness_Unadj",
                       
                       "LifeSatisf_Unadj",#Psychological Well-being 
                       #"MeanPurp_Unadj","PosAffect_Unadj",

                        "Loneliness_Unadj",#Social Relationships 
                       #"Friendship_Unadj","PercHostil_Unadj",
                       #"PercReject_Unadj","EmotSupp_Unadj","InstruSupp_Unadj",

                       "PercStress_Unadj", #Stress and Self Efficacy
                       #"SelfEff_Unadj"

                       

]

In [39]:
#MOTOR
Motor_feature_names=[
                     
                     "Endurance_AgeAdj", #Endurance (2 minute walk test)
                     "GaitSpeed_Comp", #Locomotion (4-meter walk test)
                     "Dexterity_AgeAdj", #Dexterity (9-hole Pegboard)
                     "Strength_AgeAdj", #Strength (Grip Strength Dynamometry)          
]

In [40]:
df_targets.reset_index(drop=True, inplace=True)
df_cognition = df_targets[Cognition_feature_names]
df_emotion = df_targets[Emotion_feature_names]
df_motor =df_targets[Motor_feature_names]

In [41]:
#For Jupyter: (Make this on terminal, not in Jupyter notebook cell.)
#conda install -c conda-forge category_encoders

import category_encoders as ce

df_targets.reset_index(drop=True, inplace=True)
gender_col  = df_targets[["Age","Gender"]]

encoder= ce.OrdinalEncoder(cols=['Age',"Gender"])

gender_col = encoder.fit_transform(gender_col)
gender_col.head(2)


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,Age,Gender
0,1,1
1,1,2


In [42]:
df_tda_cognition = pd.concat([df_tda,gender_col, df_cognition,df_emotion, df_motor], axis=1)
df_tda_cognition.shape

(998, 29)

### Replace NaN with mean

In [43]:
df_tda_cognition.isnull().any()

TDA_feature_1       False
TDA_feature_2       False
Age                 False
Gender              False
PicSeq_AgeAdj       False
CardSort_AgeAdj      True
Flanker_AgeAdj      False
PMAT24_A_CR          True
ReadEng_AgeAdj      False
PicVocab_AgeAdj     False
ProcSpeed_AgeAdj    False
DDisc_AUC_200        True
DDisc_AUC_40K        True
VSPLOT_TC            True
SCPT_SEN             True
SCPT_SPEC            True
IWRD_TOT             True
ListSort_AgeAdj     False
ER40_CR              True
AngHostil_Unadj      True
FearSomat_Unadj      True
Sadness_Unadj        True
LifeSatisf_Unadj     True
Loneliness_Unadj     True
PercStress_Unadj     True
Endurance_AgeAdj     True
GaitSpeed_Comp      False
Dexterity_AgeAdj    False
Strength_AgeAdj      True
dtype: bool

In [44]:
df_tda_cognition[list(df_tda_cognition.columns)] = df_tda_cognition[list(df_tda_cognition.columns)].fillna(value=df_tda_cognition[list(df_tda_cognition.columns)].mean())

df_tda_cognition.head(5)


,TDA_feature_1,TDA_feature_2,Age,Gender,PicSeq_AgeAdj,CardSort_AgeAdj,Flanker_AgeAdj,PMAT24_A_CR,ReadEng_AgeAdj,PicVocab_AgeAdj,...,AngHostil_Unadj,FearSomat_Unadj,Sadness_Unadj,LifeSatisf_Unadj,Loneliness_Unadj,PercStress_Unadj,Endurance_AgeAdj,GaitSpeed_Comp,Dexterity_AgeAdj,Strength_AgeAdj
0,6.819667,4.526492,1,1,118.78,104.94,116.55,20.0,103.4441,117.03610,...,61.7,61.2,55.0,45.6,63.8,57.8,121.27,1.24,94.23,129.43
1,6.821509,4.723720,1,2,103.45,109.92,101.90,17.0,98.7300,96.81000,...,60.8,47.2,53.4,48.0,53.7,57.9,111.10,1.58,105.21,84.59
2,6.817826,4.452232,2,1,125.19,100.77,113.51,7.0,125.6400,132.63000,...,42.8,54.7,49.9,57.7,51.9,46.8,121.80,1.51,106.24,124.24
3,6.820391,4.500798,1,1,101.69,115.18,114.18,23.0,132.4124,146.59710,...,49.1,55.3,44.2,60.1,53.5,37.8,102.79,1.10,107.85,118.90
4,6.824610,4.486151,2,2,70.00,94.30,92.33,11.0,101.1697,69.45302,...,49.0,40.1,48.7,46.5,50.3,48.8,73.07,1.24,96.00,106.93


In [45]:
df_tda_cognition.isnull().any()

TDA_feature_1       False
TDA_feature_2       False
Age                 False
Gender              False
PicSeq_AgeAdj       False
CardSort_AgeAdj     False
Flanker_AgeAdj      False
PMAT24_A_CR         False
ReadEng_AgeAdj      False
PicVocab_AgeAdj     False
ProcSpeed_AgeAdj    False
DDisc_AUC_200       False
DDisc_AUC_40K       False
VSPLOT_TC           False
SCPT_SEN            False
SCPT_SPEC           False
IWRD_TOT            False
ListSort_AgeAdj     False
ER40_CR             False
AngHostil_Unadj     False
FearSomat_Unadj     False
Sadness_Unadj       False
LifeSatisf_Unadj    False
Loneliness_Unadj    False
PercStress_Unadj    False
Endurance_AgeAdj    False
GaitSpeed_Comp      False
Dexterity_AgeAdj    False
Strength_AgeAdj     False
dtype: bool

## 4. TDA

##cover_complex.py


In [46]:
import numpy as np
import itertools
import matplotlib
import matplotlib.pyplot as plt

from sklearn.base            import BaseEstimator, TransformerMixin
from sklearn.cluster         import DBSCAN, AgglomerativeClustering
from sklearn.metrics         import pairwise_distances
from scipy.spatial.distance  import directed_hausdorff

In [47]:
class CoverComplexPy(BaseEstimator, TransformerMixin):
    """
    This is a mother class for MapperComplex, GraphInducedComplex and NerveComplex. 
    Attributes:
        simplex_tree (gudhi SimplexTree): simplicial complex representing the cover complex computed after calling the fit() method.
        node_info (dictionary): various information associated to the nodes of the cover complex. 
    """
    def __init__(self, input_name="data", cover_name="cover", color_name="color", verbose=False):
        """
        Constructor for the CoverComplexPy class.
        Parameters:
        """
        self.input_name, self.cover_name, self.color_name, self.verbose = input_name, cover_name, color_name, verbose

    def get_networkx(self, get_attrs=False):
        """
        Turn the 1-skeleton of the cover complex computed after calling fit() method into a networkx graph.
        This function requires networkx (https://networkx.org/documentation/stable/install.html).
        Parameters:
            get_attrs (bool): if True, the color functions will be used as attributes for the networkx graph.
        Returns:
            G (networkx graph): graph representing the 1-skeleton of the cover complex.
        """
        try:
            import networkx as nx
            st = self.simplex_tree
            G = nx.Graph()
            for (splx,_) in st.get_skeleton(1):	
                if len(splx) == 1:
                    G.add_node(splx[0])
                if len(splx) == 2:
                    G.add_edge(splx[0], splx[1])
            if get_attrs:
                attrs = {k: {"attr_name": self.node_info[k]["colors"]} for k in G.nodes()}
                nx.set_node_attributes(G, attrs)
            return G
        except ImportError:
            print("Networkx not found, nx graph not computed")

    def print_to_dot(self, epsv=.2, epss=.4):
        """
        Write the cover complex in a DOT file called "{self.input_name}.dot", that can be processed with, e.g., neato.
        Parameters:
            epsv (float): scale the node colors between [epsv, 1-epsv]
            epss (float): scale the node sizes between [epss, 1-epss]
        """
        st = self.simplex_tree 
        node_info = self.node_info

        maxv, minv = max([node_info[k]["colors"][0] for k in node_info.keys()]), min([node_info[k]["colors"][0] for k in node_info.keys()])
        maxs, mins = max([node_info[k]["size"]      for k in node_info.keys()]), min([node_info[k]["size"]      for k in node_info.keys()])  

        f = open(self.input_name + ".dot", "w")
        f.write("graph MAP{")
        cols = []
        for (simplex,_) in st.get_skeleton(0):
            cnode = (1.-2*epsv) * (node_info[simplex[0]]["colors"][0] - minv)/(maxv-minv) + epsv if maxv != minv else 0
            snode = (1.-2*epss) * (node_info[simplex[0]]["size"]-mins)/(maxs-mins) + epss if maxs != mins else 1
            f.write(  str(simplex[0]) + "[shape=circle width=" + str(snode) + " fontcolor=black color=black label=\""  + "\" style=filled fillcolor=\"" + str(cnode) + ", 1, 1\"]")
            cols.append(cnode)
        for (simplex,_) in st.get_filtration():
            if len(simplex) == 2:
                f.write("  " + str(simplex[0]) + " -- " + str(simplex[1]) + " [weight=15];")
        f.write("}")
        f.close()

        L = np.linspace(epsv, 1.-epsv, 100)
        colsrgb = []
        try:
            import colorsys
            for c in L:
                colsrgb.append(colorsys.hsv_to_rgb(c,1,1))
            fig, ax = plt.subplots(figsize=(6, 1))
            fig.subplots_adjust(bottom=0.5)
            my_cmap = matplotlib.colors.ListedColormap(colsrgb, name=self.color_name)
            cb = matplotlib.colorbar.ColorbarBase(ax, cmap=my_cmap, norm=matplotlib.colors.Normalize(vmin=minv, vmax=maxv), orientation="horizontal")
            cb.set_label(self.color_name)
            fig.savefig("colorbar_" + self.color_name + ".pdf", format="pdf")
            plt.close()
        except ImportError:
            print("colorsys not found, colorbar not printed")

    def print_to_txt(self):
        """
        Write the cover complex to a TXT file called "{self.input_name}.txt", that can be processed with KeplerMapper.
        """
        st = self.simplex_tree
        f = open(self.input_name + ".txt", "w")
        f.write(self.input_name + "\n")
        f.write(self.cover_name + "\n")
        f.write(self.color_name + "\n")
        f.write("0 0\n")
        f.write(str(st.num_vertices()) + " " + str(len(list(st.get_skeleton(1)))-st.num_vertices()) + "\n")
        name2id = {}
        idv = 0
        for s,_ in st.get_skeleton(0):
            f.write(str(idv) + " " + str(self.node_info[s[0]]["colors"][0]) + " " + str(self.node_info[s[0]]["size"]) + "\n")
            name2id[s[0]] = idv
            idv += 1
        for s,_ in st.get_skeleton(1):
            if len(s) == 2:
                f.write(str(name2id[s[0]]) + " " + str(name2id[s[1]]) + "\n")
        f.close()

    class _constant_clustering():
        def fit_predict(X):
            return np.zeros([len(X)], dtype=np.int32)


class MapperComplex(CoverComplexPy, BaseEstimator, TransformerMixin):
    """
    This is a class for computing Mapper simplicial complexes on point clouds or distance matrices.
    """
    def __init__(self, input_type="point cloud", colors=None, mask=0,
                       filters=None, filter_bnds=None, resolutions=None, gains=None, clustering=DBSCAN(), N=100, beta=0., C=10.,
                       input_name="data", cover_name="cover", color_name="color", verbose=False):
        """
        Constructor for the MapperComplex class.
        Parameters:
            input_type (string): type of input data. Either "point cloud" or "distance matrix".
            colors (numpy array of shape (num_points) x (num_colors)): functions used to color the nodes of the cover complex. More specifically, coloring is done by computing the means of these functions on the subpopulations corresponding to each node. If None, first coordinate is used if input is point cloud, and eccentricity is used if input is distance matrix.
            mask (int): threshold on the size of the cover complex nodes (default 0). Any node associated to a subpopulation with less than **mask** points will be removed.
            filters (numpy array of shape (num_points) x (num_filters)): filter functions (sometimes called lenses) used to compute the cover. Each column of the numpy array defines a scalar function defined on the input points.
            filter_bnds (numpy array of shape (num_filters) x 2): limits of each filter, of the form [[f_1^min, f_1^max], ..., [f_n^min, f_n^max]]. If one of the values is numpy.nan, it can be computed from the dataset with the fit() method.
            resolutions (numpy array of shape num_filters containing integers): resolution of each filter function, ie number of intervals required to cover each filter image. If None, it is estimated from data.
            gains (numpy array of shape num_filters containing doubles in [0,1]): gain of each filter function, ie overlap percentage of the intervals covering each filter image.
            clustering (class): clustering class (default sklearn.cluster.DBSCAN()). Common clustering classes can be found in the scikit-learn library (such as AgglomerativeClustering for instance). If None, it is set to hierarchical clustering, with scale estimated from data.
            N (int): subsampling iterations (default 100) for estimating scale and resolutions. Used only if clustering or resolutions = None. See http://www.jmlr.org/papers/volume19/17-291/17-291.pdf for details.
            beta (double): exponent parameter (default 0.) for estimating scale and resolutions. Used only if clustering or resolutions = None. See http://www.jmlr.org/papers/volume19/17-291/17-291.pdf for details.
            C (double): constant parameter (default 10.) for estimating scale and resolutions. Used only if clustering or resolutions = None. See http://www.jmlr.org/papers/volume19/17-291/17-291.pdf for details.
            input_name (string): name of dataset. Used when generating plots.
            cover_name (string): name of cover. Used when generating plots.
            color_name (string): name of color function. Used when generating plots.
            verbose (bool): whether to display info while computing.
        """
        self.filters, self.filter_bnds, self.resolutions, self.gains, self.colors, self.clustering = filters, filter_bnds, resolutions, gains, colors, clustering
        self.input_type, self.mask, self.N, self.beta, self.C = input_type, mask, N, beta, C
        CoverComplexPy.__init__(self, input_name, cover_name, color_name, verbose)

    def estimate_scale(self, X, N=100, inp="point cloud", beta=0., C=10.):
        """
        Compute estimated scale of a point cloud or a distance matrix.
        Parameters:
            X (numpy array of shape (num_points) x (num_coordinates) if point cloud and (num_points) x (num_points) if distance matrix): input point cloud or distance matrix.
            N (int): subsampling iterations (default 100). See http://www.jmlr.org/papers/volume19/17-291/17-291.pdf for details.
            inp (string): either "point cloud" or "distance matrix". Type of input data (default "point cloud").
            beta (double): exponent parameter (default 0.). See http://www.jmlr.org/papers/volume19/17-291/17-291.pdf for details.
            C (double): constant parameter (default 10.). See http://www.jmlr.org/papers/volume19/17-291/17-291.pdf for details.
        Returns:
            delta (double): estimated scale that can be used with eg agglomerative clustering.
        """
        num_pts = X.shape[0]
        delta, m = 0., int(  num_pts / np.exp((1+beta) * np.log(np.log(num_pts)/np.log(C)))  )
        for _ in range(N):
            subpop = np.random.choice(num_pts, size=m, replace=False)
            if inp == "point cloud":
                d, _, _ = directed_hausdorff(X, X[subpop,:])
            if inp == "distance matrix":
                d = np.max(np.min(X[:,subpop], axis=1), axis=0)
            delta += d/N
        return delta

    def get_optimal_parameters_for_agglomerative_clustering(self, X, beta=0., C=10., N=100):
        """
        Compute optimal scale and resolutions for a point cloud or a distance matrix.
        Parameters:
            X (numpy array of shape (num_points) x (num_coordinates) if point cloud and (num_points) x (num_points) if distance matrix): input point cloud or distance matrix.
            beta (double): exponent parameter (default 0.). See http://www.jmlr.org/papers/volume19/17-291/17-291.pdf for details.
            C (double): constant parameter (default 10.). See http://www.jmlr.org/papers/volume19/17-291/17-291.pdf for details.
            N (int): subsampling iterations (default 100). See http://www.jmlr.org/papers/volume19/17-291/17-291.pdf for details.
        Returns:
            delta (double): optimal scale that can be used with agglomerative clustering.
            resolutions (numpy array of shape (num_filters): optimal resolutions associated to each filter.
        """
        num_filt, delta = self.filters.shape[1], 0
        delta = self.estimate_scale(X=X, N=N, inp=self.input_type, C=C, beta=beta)

        pairwise = pairwise_distances(X, metric="euclidean") if self.input_type == "point cloud" else X
        pairs = np.argwhere(pairwise <= delta)
        num_pairs = pairs.shape[0]
        res = []
        for f in range(num_filt):
            F = self.filters[:,f]
            resf = 0
            for p in range(num_pairs):
                resf = max(resf, abs(F[pairs[p,0]] - F[pairs[p,1]]))
            res.append(resf)

        return delta, np.array(res)

    def fit(self, X, y=None):
        """
        Fit the MapperComplex class on a point cloud or a distance matrix: compute the Mapper complex and store it in a simplex tree called simplex_tree.
        Parameters:
            X (numpy array of shape (num_points) x (num_coordinates) if point cloud and (num_points) x (num_points) if distance matrix): input point cloud or distance matrix.
            y (n x 1 array): point labels (unused).
        """
        if self.filters is None:
            if self.input_type == "point cloud":
                self.filters = X[:,0:1]
            elif self.input_type == "distance matrix":
                self.filters = X.max(axis=0)[:,None]
        if self.colors is None:
            if self.input_type == "point cloud":
                self.colors = X[:,0:1]
            elif self.input_type == "distance matrix":
                self.colors = X.max(axis=0)[:,None]

        num_pts, num_filters = self.filters.shape[0], self.filters.shape[1]

        # If some filter limits are unspecified, automatically compute them
        if self.filter_bnds is None: 
            self.filter_bnds = np.hstack([np.min(self.filters, axis=0)[:,np.newaxis], np.max(self.filters, axis=0)[:,np.newaxis]])

        # If some resolutions are not specified, automatically compute them
        if self.gains is None:
            self.gains = .33 * np.ones(num_filters)
        if self.resolutions is None or self.clustering is None:
            delta, resolutions = self.get_optimal_parameters_for_agglomerative_clustering(X=X, beta=self.beta, C=self.C, N=self.N)
            if self.clustering is None:
                if self.input_type == "point cloud":
                    self.clustering = AgglomerativeClustering(n_clusters=None, linkage="single", distance_threshold=delta, affinity="euclidean")  
                else:
                    self.clustering = AgglomerativeClustering(n_clusters=None, linkage="single", distance_threshold=delta, affinity="precomputed")
            if self.resolutions is None:
                self.resolutions = np.multiply(resolutions, 1./self.gains)
                self.resolutions = np.array([int( (self.filter_bnds[ir,1]-self.filter_bnds[ir,0])/r) for ir, r in enumerate(self.resolutions)])

        # Initialize attributes
        self.simplex_tree, self.node_info = SimplexTree(), {}

        if np.all(self.gains < .5):
            
            # Compute which points fall in which patch or patch intersections
            interval_inds, intersec_inds = np.empty(self.filters.shape), np.empty(self.filters.shape)
            for i in range(num_filters):
                f, r, g = self.filters[:,i], self.resolutions[i], self.gains[i]
                min_f, max_f = self.filter_bnds[i,0], np.nextafter(self.filter_bnds[i,1], np.inf)
                interval_endpoints, l = np.linspace(min_f, max_f, num=r+1, retstep=True)
                intersec_endpoints = []
                for j in range(1, len(interval_endpoints)-1):
                    intersec_endpoints.append(interval_endpoints[j] - g*l / (2 - 2*g))
                    intersec_endpoints.append(interval_endpoints[j] + g*l / (2 - 2*g))
                interval_inds[:,i] = np.digitize(f, interval_endpoints)
                intersec_inds[:,i] = 0.5 * (np.digitize(f, intersec_endpoints) + 1)

            # Build the binned_data map that takes a patch or a patch intersection and outputs the indices of the points contained in it
            binned_data = {}
            for i in range(num_pts):
                list_preimage = []
                for j in range(num_filters):
                    a, b = interval_inds[i,j], intersec_inds[i,j]
                    list_preimage.append([a])
                    if b == a:
                        list_preimage[j].append(a+1)
                    if b == a-1:
                        list_preimage[j].append(a-1)
                list_preimage = list(itertools.product(*list_preimage))
                for pre_idx in list_preimage:
                    try:
                        binned_data[pre_idx].append(i)
                    except KeyError:
                        binned_data[pre_idx] = [i]

        else:

            # Compute interval endpoints for each filter
            l_int, r_int = [], []
            for i in range(num_filters):
                L, R = [], []
                f, r, g = self.filters[:,i], self.resolutions[i], self.gains[i]
                min_f, max_f = self.filter_bnds[i,0], np.nextafter(self.filter_bnds[i,1], np.inf)
                interval_endpoints, l = np.linspace(min_f, max_f, num=r+1, retstep=True)
                for j in range(len(interval_endpoints)-1):
                    L.append(interval_endpoints[j]   - g*l / (2 - 2*g))
                    R.append(interval_endpoints[j+1] + g*l / (2 - 2*g))
                l_int.append(L)
                r_int.append(R)

            # Build the binned_data map that takes a patch or a patch intersection and outputs the indices of the points contained in it
            binned_data = {}
            for i in range(num_pts):
                list_preimage = []
                for j in range(num_filters):
                    fval = self.filters[i,j]
                    start, end = int(min(np.argwhere(np.array(r_int[j]) >= fval))), int(max(np.argwhere(np.array(l_int[j]) <= fval)))
                    list_preimage.append(list(range(start, end+1)))
                list_preimage = list(itertools.product(*list_preimage))
                for pre_idx in list_preimage:
                    try:
                        binned_data[pre_idx].append(i)
                    except KeyError:
                        binned_data[pre_idx] = [i]

        # Initialize the cover map, that takes a point and outputs the clusters to which it belongs
        cover, clus_base = [[] for _ in range(num_pts)], 0

        # For each patch
        for preimage in binned_data:

            # Apply clustering on the corresponding subpopulation
            idxs = np.array(binned_data[preimage])
            if len(idxs) > 1:
                clusters = self.clustering.fit_predict(X[idxs,:]) if self.input_type == "point cloud" else self.clustering.fit_predict(X[idxs,:][:,idxs])
            elif len(idxs) == 1:
                clusters = np.array([0])
            else:
                continue

            # Collect various information on each cluster
            num_clus_pre = np.max(clusters) + 1
            for clus_i in range(num_clus_pre):
                node_name = clus_base + clus_i
                subpopulation = idxs[clusters == clus_i]
                self.node_info[node_name] = {}
                self.node_info[node_name]["indices"] = subpopulation
                self.node_info[node_name]["size"] = len(subpopulation)
                self.node_info[node_name]["colors"] = np.mean(self.colors[subpopulation,:], axis=0)
                self.node_info[node_name]["patch"] = preimage

            # Update the cover map
            for pt in range(clusters.shape[0]):
                node_name = clus_base + clusters[pt]
                if clusters[pt] != -1 and self.node_info[node_name]["size"] >= self.mask:
                    cover[idxs[pt]].append(node_name)

            clus_base += np.max(clusters) + 1

        # Insert the simplices of the Mapper complex 
        for i in range(num_pts):
            self.simplex_tree.insert(cover[i])

        return self


In [48]:
!pip install gudhi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
#from gudhi.sklearn import MapperComplex, GraphInducedComplex, NerveComplex
#from cover_complex import MapperComplex, GraphInducedComplex, NerveComplex
#Instead of above, we define the MapperComplex in a cell in this notebook above

from gudhi import bottleneck_distance
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook

##Filter for Mapper:

In [50]:
df_tda_cognition_np = df_tda_cognition.to_numpy()
df_tda_cognition_np.shape

(998, 29)

1. L_P norm:

In [51]:
#Define filter function for Mapper:
L_p_filter = np.zeros((998,))
print(L_p_filter.shape)

k=2
p=5

for patients in range(0,len(df_tda_cognition_np)):
  summ=0
  filter_temp = 0
  for features in range(len(df_tda_cognition_np[patients])):
    summ += (df_tda_cognition_np[patients][features])**p
    #print(df_tda_cognition_np[patients][features])
  filter_temp = summ**(k/p)
  L_p_filter[patients] = filter_temp

(998,)


L_infinity Centrality:

In [52]:
'''
from scipy.spatial import distance

L_infinity_filter = np.zeros((998,))

for i in range(0,len(df_tda_cognition_np)):
  max_distance=0

  for j in range(len(df_tda_cognition_np)):
    if distance.euclidean(df_tda_cognition_np[i], df_tda_cognition_np[j]) > max_distance:
      max_distance = distance.euclidean(df_tda_cognition_np[i], df_tda_cognition_np[j])

  L_infinity_filter[i] = max_distance
'''

'\nfrom scipy.spatial import distance\n\nL_infinity_filter = np.zeros((998,))\n\nfor i in range(0,len(df_tda_cognition_np)):\n  max_distance=0\n\n  for j in range(len(df_tda_cognition_np)):\n    if distance.euclidean(df_tda_cognition_np[i], df_tda_cognition_np[j]) > max_distance:\n      max_distance = distance.euclidean(df_tda_cognition_np[i], df_tda_cognition_np[j])\n\n  L_infinity_filter[i] = max_distance\n'

In [53]:
#L-infinity centrality, which assigns to each point the distance to the point most distant from it. 

#L-infinity centrality is defined for each data point y to be the maximum distance from y to any other data point in the 
#data set. It produces a more detailed and succinct description of the data set than a typical scatter plots display. 
#Large values of this function correspond to points that are far from the center of the data set. 
from scipy.spatial.distance import pdist, squareform

#pairwise_dist = squareform(pdist(df_tda_cognition_np, 'euclidean')) 
pairwise_dist = squareform(pdist(df_tda_cognition_np, 'correlation'))

L_infinity = np.amax(pairwise_dist, axis = 1)
L_infinity.shape, L_infinity[:10]

((998,), array([0.09952781, 0.08583295, 0.10785893, 0.11864991, 0.10944291,
        0.09198246, 0.11137649, 0.11445129, 0.11242627, 0.11012782]))

In [54]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=1, n_iter=7, random_state=42)
SVD_filter = svd.fit_transform(df_tda_cognition_np)
print(SVD_filter.shape)


(998, 1)


In [55]:
'''
The original approach from Lum et al. is:

lens1: The target variable (survival or relapse)
lens2: L-infinity centrality

Lens1 then causes the layout to separate on the target. 
Lens2 gives a centrality measure. You can 
then find the different types of survival or relapse, and flares should form more clearly (each signifying 
a different type of departure from the centrality).
'''

#target_col = df_tda_cognition["ListSort_AgeAdj"].to_numpy()

#ff= np.concatenate(( L_infinity[:,np.newaxis], target_col[:,np.newaxis] ), axis=1)
#ff= np.concatenate(( SVD_filter, L_infinity_filter[:,np.newaxis] ), axis=1)
#ff= np.concatenate(( L_infinity_filter[:,np.newaxis], SVD_filter ), axis=1)
#ff= np.concatenate(( L_p_filter[:,np.newaxis], L_infinity_filter[:,np.newaxis] ), axis=1)

#ff.shape

'\nThe original approach from Lum et al. is:\n\nlens1: The target variable (survival or relapse)\nlens2: L-infinity centrality\n\nLens1 then causes the layout to separate on the target. \nLens2 gives a centrality measure. You can \nthen find the different types of survival or relapse, and flares should form more clearly (each signifying \na different type of departure from the centrality).\n'

##Helper fuctions

In [56]:
from scipy.sparse.csgraph    import dijkstra, shortest_path, connected_components
from scipy.stats             import ks_2samp

def find(i, parents):
    if parents[i] == i:
        return i
    else:
        return find(parents[i], parents)

def union(i, j, parents, f):
    if f[i] <= f[j]:
        parents[j] = i
    else:
        parents[i] = j
        
def compute_topological_features(M, threshold=0.):
    """
    Compute the topological features (connected components, up/down branches, loops) of 
    the 1-skeleton of the cover complex. Connected components and loops are computed with scipy 
    functions, and branches are detected with Union-Find and 0-dimensional persistence of the 1-skeleton.
    Parameters:
        threshold (float): any topological feature whose size is less than this parameter 
        (relative to the first color function) will be discarded.
    Returns:
        dgm (list of (dim,(a,b)) tuples): list of feature characteristics. dim is the 
        topological dimension of the feature (0 for CCs and branches, 1 for loops), a,b are the min and max 
        of the first color function along the feature.
        bnds (list of lists): list of feature points. Each element of this list is the list of point IDs 
        forming the corresponding feature. 
    """
    st = M.simplex_tree
    num_nodes = st.num_vertices()
    function, namefunc, invnamefunc = {}, {}, {}
    nodeID = 0
    for (s,_) in st.get_skeleton(0):
        namefunc[s[0]] = nodeID
        invnamefunc[nodeID] = s[0]
        function[s[0]] = M.node_info[s[0]]["colors"][0]
        nodeID += 1
    dgm, bnd = [], []

    # connected_components
    A = np.zeros([num_nodes, num_nodes])
    for (splx,_) in st.get_skeleton(1):
        if len(splx) == 2:
            A[namefunc[splx[0]], namefunc[splx[1]]] = 1
            A[namefunc[splx[1]], namefunc[splx[0]]] = 1
    _, ccs = connected_components(A, directed=False)
    for ccID in np.unique(ccs):
        pts = np.argwhere(ccs == ccID).flatten()
        vals = [function[invnamefunc[p]] for p in pts]
        if np.abs(min(vals) - max(vals)) >= threshold:
            dgm.append((0, (min(vals), max(vals))))
            bnd.append([invnamefunc[p] for p in pts])

    # loops
    G = M.get_networkx()
    try:
        from networkx import cycle_basis
        bndall = cycle_basis(G)
        for pts in bndall:
            vals = [function[p] for p in pts]
            if np.abs(min(vals) - max(vals)) >= threshold:	
                dgm.append((1,(min(vals), max(vals))))
                bnd.append(pts)
    except ImportError:
        print("Networkx not found, loops not computed")
        
    # branches
    for topo_type in ["downbranch", "upbranch"]:

        lfunction = []
        for i in range(num_nodes):
            lfunction.append(function[invnamefunc[i]])

        # upranch is downbranch of opposite function
        if topo_type == "upbranch":
            lfunction = [-f for f in lfunction]

        # sort vertices according to function values and compute inverse function 
        sorted_idxs = np.argsort(np.array(lfunction))
        inv_sorted_idxs = np.zeros(num_nodes)
        for i in range(num_nodes):
            inv_sorted_idxs[sorted_idxs[i]] = i

        # go through all vertices in ascending function order
        persistence_diag, persistence_set, parents, visited = {}, {}, -np.ones(num_nodes, dtype=np.int32), {}
        for i in range(num_nodes):

            current_pt = sorted_idxs[i]
            neighbors = np.ravel(np.argwhere(A[current_pt,:] == 1))
            lower_neighbors = [n for n in neighbors if inv_sorted_idxs[n] <= i] if len(neighbors) > 0 else []

            # no lower neighbors: current point is a local minimum
            if lower_neighbors == []:
                parents[current_pt] = current_pt

            # some lower neighbors exist
            else:

                # find parent pg of lower neighbors with lowest function value
                neigh_parents = [find(n, parents) for n in lower_neighbors]
                pg = neigh_parents[np.argmin([lfunction[n] for n in neigh_parents])]

                # set parent of current point to pg
                parents[current_pt] = pg

                # for each lower neighbor, we will create a persistence diagram point and corresponding set of nodes
                for neighbor in lower_neighbors:

                    # get parent pn
                    pn = find(neighbor, parents)
                    val = lfunction[pn]
                    persistence_set[pn] = []

                    # we will create persistence set only if parent pn is not local minimum pg
                    if pn != pg:
                        # go through all strictly lower nodes with parent pn
                        for v in sorted_idxs[:i]:
                            if find(v, parents) == pn:
                                # if it is already part of another persistence set, continue
                                try:
                                    visited[v]
                                # else, mark visited and include it in current persistence set
                                except KeyError:
                                    visited[v] = True
                                    persistence_set[pn].append(v)

                        # add current point to persistence set
                        persistence_set[pn].append(current_pt)

                        # do union and create persistence point corresponding to persistence set if persistence is sufficiently large
                        if np.abs(lfunction[pn]-lfunction[current_pt]) >= threshold:
                            persistence_diag[pn] = current_pt
                            union(pg, pn, parents, lfunction)

        for key, val in iter(persistence_diag.items()):
            if topo_type == "downbranch":
                dgm.append((0, (lfunction[key],  lfunction[val])))
            elif topo_type == "upbranch":
                dgm.append((0, (-lfunction[val], -lfunction[key])))
            bnd.append([invnamefunc[v] for v in persistence_set[key]])

    bnd = [list(b) for b in bnd]
    M.persistence_diagram, M.persistence_sets = dgm, bnd 
    return dgm, bnd

def bootstrap_topological_features(M, N):
    """
    Use bootstrap to empirically assess stability of the features. This function computes a distribution of 
    bottleneck distances, that can used afterwards to run tests on each topological feature.
    Parameters:
        N (int): number of bootstrap iterations.
    """

    dgm = M.persistence_diagram
    num_pts, distribution = len(M.data), []
    for bootstrap_id in range(N):

        print(str(bootstrap_id) + "th iteration")

        # Randomly select points
        idxs = np.random.choice(num_pts, size=num_pts, replace=True)
        Xboot = M.data[idxs,:] if M.input_type == "point cloud" else M.data[idxs,:][:,idxs]
        f_boot, c_boot = M.filters[idxs,:], M.colors[idxs,:]
        Mboot = M.__class__(filters=f_boot, filter_bnds=M.filter_bnds, colors=c_boot, 
                            resolutions=M.resolutions, gains=M.gains, 
                            input_type=M.input_type, clustering=M.clustering).fit(Xboot)

        # Compute the corresponding persistence diagrams
        dgm_boot, _ = compute_topological_features(Mboot)

        # Compute the bottleneck distance
        npts, npts_boot = len(dgm), len(dgm_boot)
        D1 = np.array([[dgm[pt][1][0], dgm[pt][1][1]] for pt in range(npts)]) 
        D2 = np.array([[dgm_boot[pt][1][0], dgm_boot[pt][1][1]] for pt in range(npts_boot)])
        bottle = bottleneck_distance(D1, D2)
        distribution.append(bottle)
        M.distribution = np.sort(distribution)

def get_distance_from_confidence_level(M, alpha=.95, complex_type='mapper'):
    """
    Compute the bottleneck distance threshold corresponding to a specific confidence level.
    Parameters:
        alpha (float): confidence level.
    Returns:
        distance value (float); each feature whose size is above this distance is sure at confidence level alpha.
    """
    return M.distribution[int(alpha*len(M.distribution))]

def get_confidence_level_from_distance(M, distance):
    """
    Compute the confidence level of a specific bottleneck distance threshold.
    Parameters:
        distance (float): bottleneck distance threshold.
    Returns:
        confidence level (float); each feature whose size is above the distance threshold is sure at 
        this confidence level.
    """
    return len(np.argwhere(M.distribution <= distance))/len(M.distribution)

def get_pvalue(M):
    """
    Compute the p-value, i.e. the opposite of the confidence level of the largest bottleneck distance 
    preserving the topological features.
    Returns:
        p-value (float)
    """
    distancemin = min([np.abs(pt[1][0]-pt[1][1]) for pt in M.persistence_diagram])
    return 1.-M.compute_confidence_from_distance(distancemin)

from scipy.stats import ks_2samp, ks_1samp
from scipy import stats

def compute_differential_coordinates(M, nodes=None, features=None, sparse=False):
    """
    Compute the coordinates that best explain a set of nodes VS the rest of the nodes 
    (in the 1-skeleton of the cover complex) with a Kolmogorov-Smirnov test. 
    Only works if input_type is "point cloud".
    Parameters:
        nodes (list of integers): list of nodes to try. For instance, one can take the list of 
        nodes obtained after calling "compute_topological_features"
        features (list of integers): the coordinates to try. All coordinates are tested if None.
        sparse (bool): set to True if your data is sparse and there will be speedup, otherwise use False.
    Returns:
        features (list of integers): the list of coordinates, ranked from smallest to largest p-values.
        p-values (list of float): the corresponding p-values. 
    """
    if M.input_type == "distance matrix":
        print("Need coordinates for running differential coordinates!")
        raise

    node_info = M.node_info
    X = M.data
    nodes = [s[0] for s,_ in self.simplex_tree.get_skeleton(0)] if nodes is None else nodes

    if features is None:
        features = np.arange(X.shape[1])

    list_idxs1 = list(np.unique(np.concatenate([node_info[node_name]["indices"] for node_name in nodes])))
    list_idxs2 = list(set(np.arange(X.shape[0]))-set(list_idxs1))

    #print(len(list_idxs1), len(list_idxs2))
    pvals = []
    for f in features:
        if sparse:
            Xsp = csr_matrix(X)
            group1 = np.squeeze(np.array(Xsp[list_idxs1,f].todense()))
            group2 = np.squeeze(np.array(Xsp[list_idxs2,f].todense()))
        else:
            group1, group2 = X[list_idxs1,f], X[list_idxs2,f]
        
        if len(group1)==0:
          _,pval = ks_1samp(group2, stats.norm.cdf)

        elif len(group2)==0:
          _,pval = ks_1samp(group1, stats.norm.cdf)
        else:
          _,pval = ks_2samp(group1, group2)
        pvals.append(pval)
    pvals = np.array(pvals)
    F, P = features[np.argsort(pvals)], np.sort(pvals) 
    return F, P

##Define Mapper:

## Complex computation

In [57]:
#4,15,0.15,0.05

The cover complex can now be computed in a single line of code!

In [58]:
from gudhi import SimplexTree, CoverComplex

#_ = cover_complex.fit(df_tda_cognition_np)

## Visualization

# Topological features

###Community Detection


In [59]:
def pvalue_check( meanings_dictionary, bnd_temp, parameters, numm,df_tda_cognition):
  print("P-VALUE CHECK")

  # Print them with feature names from most important to least important:
  f = open("output"+str(numm)+".txt", "a")
  print(df_tda_cognition.shape, file=f)
  print("PARAMETERS =", file=f)
  print("resolution1 =", parameters[0], file=f)
  print("gain1 =", parameters[1]/100, file=f)
  print("resolution2 =", parameters[2], file=f)
  print("gain2 =", parameters[3]/100, file=f)
  print("Clustering =", parameters[4], file=f)
  print("LENS1 =", parameters[5], file=f)
  print("LENS2 =", parameters[6], file=f)


  for i in range(len(bnd_temp)):
    cog=[]
    
    print("--------------------------------------------------------------",file=f)
    print("COMMUNITY-",i, file=f)
    #print("COMMUNITY-",i,)
    print("------------",file=f)
    coordinates, pvalues = compute_differential_coordinates(cover_complex, nodes=bnd_temp[i])
    #print(bnd_temp[i])
    #print("Coordinates:")
    #print(coordinates_and_pvalues[0])
    #print()
    #print("Feature names that creates this community:")
    featuress= [df_tda_cognition.columns[j] for j in coordinates]
    #print([df_tda_cognition.columns[i] for i in coordinates_and_pvalues[0]] )
    #print()
    #print("Corresponding p-values:")
    #print(coordinates_and_pvalues[1])

    #Check if p_value < 0.05
    for j in range(0,len(featuress)):
      if pvalues[j] <= 0.0001:
        cog.append(meanings_dictionary[featuress[j]])

    for k in range(0,len(cog)):
      print(cog[k], file=f)
      #print(cog[k])
    #print("--------------------------------------------------------------")

  f.close()


##Visualization search

In [60]:
import sys
from PIL import Image

def concetanete_results(img1, img2,concat_imgs_path):
    images = [Image.open(x) for x in [img1,img2 ]]
    widths, heights = zip(*(i.size for i in images))

    total_width = sum(widths)
    max_height = max(heights)

    new_im = Image.new('RGB', (total_width, max_height))

    x_offset = 0
    for im in images:
      new_im.paste(im, (x_offset,0))
      x_offset += im.size[0]

    new_im.save(concat_imgs_path)

In [61]:

#meanings_dictionary
def get_features_that_makes_a_community(min_num_points_in_a_community, bnd):
  bnd_temp= []  
  #Detect communities(topological feautes) that have # of points >= min_num_points_in_a_community
  for i in range(len(bnd)):
    if len(bnd[i]) >= min_num_points_in_a_community:
      bnd_temp.append(bnd[i])
  
  return bnd_temp




## 1. DBSCAN( )


In [62]:
df_tda_cognition.shape

(998, 29)

In [63]:
#target_col = df_tda_cognition["ListSort_AgeAdj"].to_numpy()
#SVD_filter
#L_infinity 
#L_p_filter

gender_col = df_tda_cognition["Gender"].to_numpy()
memory_col = df_tda_cognition["ListSort_AgeAdj"].to_numpy()
#ff= np.concatenate(( L_infinity[:,np.newaxis], target_col[:,np.newaxis] ), axis=1)

ff = np.concatenate((gender_col[:,np.newaxis], L_infinity[:,np.newaxis] ), axis=1)              


In [64]:
from sklearn.cluster import DBSCAN, AgglomerativeClustering
import networkx as nx
#5,15,.15,.05
numm = len(range(3,4)) * len(range(2,7)) * len(range(3,7)) * len(range(2,7))


for resolution1 in range(2,3):  #2-10
    for gain1 in range(1,2):  #/100 #1-8
      for resolution2 in range(19,20):   #2-10
          for gain2 in range(2,3): #1-8   #/100
              print(resolution1, gain1/100, resolution2, gain2/100)

              #FILTER:
              filter_name="Gender"+"L_inf"

              cover_complex = MapperComplex(
                input_type='point cloud', 
                colors = df_tda_cognition["ListSort_AgeAdj"].to_numpy()[:,np.newaxis],
                mask=0,
                
                #clustering = None,
                clustering=DBSCAN(eps = 0.5, min_samples=5, metric ="correlation"),
                #clustering = AgglomerativeClustering(n_clusters=None, linkage="complete", 
                                                    #distance_threshold=0.5, affinity="cosine"),
                
                N=100, beta=0., C=10, 
                #filters = L_infinity_filter[:,np.newaxis],
                #filters = L_p_filter[:,np.newaxis], 
                filters = ff,
                
                filter_bnds=None, 
                
                #resolutions = Number of intervals required to cover each filter image
                #When you increase this, num of communities increase
                resolutions = np.array([resolution1, resolution2]), # When you decrease this, num of data points decrease. 90

                #gains = Overlap percentage of the intervals covering each filter image.
                gains=np.array([gain1/1000, gain2/100]),  #When you decrease this, data-points lose connections 0.91
                input_name="human", cover_name="coord2", color_name="coord2", verbose=True)

              parameters=[resolution1,gain1, resolution2, gain2, cover_complex.clustering,"L_infinity_filter", "SVD_filter"]
              file_name=str(numm)+"-"+str(cover_complex.clustering)+"-"+str(resolution1)+"-"+str(gain1/100)+"-"+str(resolution2)+"-"+str(gain2/100)+"-"+filter_name
              #Complex computation:
              _ = cover_complex.fit(df_tda_cognition_np)
              
              #Visualization
              G = cover_complex.get_networkx()

              plt.figure()
              nx.draw(G, pos=nx.kamada_kawai_layout(G), node_color=[cover_complex.node_info[v]["colors"][0] for v in G.nodes()])
              #plt.show()
              #result1_path = './DBSCAN(eps=0.5,min_samples=5,metric="cosine")/'+'VIS/'+'VIS_res='+str(resolution1)+'_'+'gain='+str(gain1/100)+'.png'
              plt.savefig("visualization")
              
              #Topological Features
              cover_complex.data = df_tda_cognition_np
              
              dgm, bnd = compute_topological_features(cover_complex, threshold=0.)

              #Get communities:
              min_num_points_in_a_community = 4
              bnd_temp= get_features_that_makes_a_community(min_num_points_in_a_community,bnd)
              
              
              if flag==0:
                print("NUM COMM == 0")
                continue
              if len(bnd_temp)>9:
                print("NUM COMM > 9")
                continue
              if len(bnd_temp)<2:
                print("NUM COMM < 2")
                continue
              #Visualize communities:
              G = cover_complex.get_networkx()
              plt.figure(figsize=(10,10))
              for idx, bd in enumerate(bnd_temp):
                  #print(idx,bd)
                  #plt.subplot(1,len(bnd),idx+1)
                  plt.subplot(3,3,idx+1)
                  nx.draw(G, pos=nx.kamada_kawai_layout(G), 
                          node_color=[1 if node in bd else 0 for node in G.nodes()], node_size=15)

              #plt.show()
              plt.savefig("communities")

              #Concetante results:
              concetanete_results("visualization.png","communities.png", "concat_imgs_path"+file_name+".pdf")
              
              #print(len(bnd_temp))
              pvalue_check( meanings_dictionary ,bnd_temp, parameters, file_name,df_tda_cognition)
              
              from fpdf import FPDF
              pdf = FPDF()  
              pdf.add_page()
              pdf.set_font("Arial", size = 8)
              f2 = open("output"+file_name+".txt", "r")
              
              # insert the texts in pdf
              for x in f2:
                  pdf.cell(200, 4, txt = x, ln = 1, align = 'C')
                
              # save the pdf with name .pdf
              pdf.output("deneme"+file_name+".pdf")

              from PyPDF2 import PdfFileMerger, PdfFileReader
              merger = PdfFileMerger()

              merger.append(PdfFileReader(open("concat_imgs_path"+file_name+".pdf", 'rb')))
              merger.append(PdfFileReader(open("deneme"+file_name+".pdf", 'rb')))

              merger.write("/content/drive/MyDrive/a_results/"+"000-aaa-"+file_name+".pdf")

              numm -=1


        

2 0.01 19 0.02


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

P-VALUE CHECK


In [65]:
iimport glob, os
from PyPDF2 import PdfMerger
os.chdir("/content/drive/MyDrive/a_results/2-DBSCAN_Li_SVD/")
merger = PdfMerger()

pdfs = []
for file in glob.glob("*.pdf"):
    pdfs.append(file)

for i in range(0,len(pdfs)):
    
    merger.append(pdfs[i])

merger.write("-0000x000- RESULT-old_col.pdf")
merger.close()

SyntaxError: ignored